 # Alle Karteringen omzetten

 Dit notebook voert alle karteringen via veg2hab naar een habitattypekartering. Instructies:

 - Kopieer de provinciefolders uit de OneDrive: `Spheer AI/Spheer AI - General/Projecten/Veg_2_Hab/Data/Vegetatiekarteringen` naar `testing/vegetatiekarteringen` (of naar iets anders, dan moet je `originele_karteringen_prefix` aanpassen onder **config**)
 - Eerst moeten sommige shapefiles opgeschoond worden. Dit kan met het `handmatig_opschonen_shapefiles.ipynb` notebook. Run deze als dat niet al eens gedaan is.
 - Zorg dat de paden `originele_karteringen_prefix` en `opgeschoonde_shapefiles_prefix` kloppen en overeenkomen met dezelfde paden in het `handmatig_opschonen_shapefiles.ipynb`. Deze prefixes zullen voor de paden vermeld in de spreadsheets geplakt worden.
 - Run de cellen. Deze zullen:
   - Imports importen en het bestaan van de paden controleren.
   - De voor het omzetten benodigde klassen instantieren; de was-wordt lijst, de definitietabel en de FGR kaart.
   - Spreadsheets met daarin configuraties voor iedere kartering inlezen.
   - Alle karteringen inladen in een grote dict.
   - De volgende stappen een voor een op iedere kartering uitvoeren:
     - De was-wordt lijst toepassen (toevoegen van VvN codes aan karteringen met enkel SBB).
     - De definitietabel toepassen (het opstellen van habitatvoorstellen voor ieder vlak).
     - De mitsen checken (eventuele criteria in de habitatvoorstellen checken) en voor ieder vegetatietype een habitatkeuze maken.
     - De kartering formatten als een habitattypekartering en wegschrijven.

 ### Imports etc

In [1]:
from pathlib import Path
import pandas as pd
import geopandas as gpd
import sys
sys.path.append('../')

from veg2hab.waswordtlijst import WasWordtLijst, opschonen_waswordtlijst
from veg2hab.definitietabel import DefinitieTabel, opschonen_definitietabel
from veg2hab.vegkartering import Kartering
from veg2hab.fgr import FGR
from veg2hab.io.cli import CLIInterface

CLIInterface.get_instance()

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 8)

originele_karteringen_prefix = '../testing/vegetatiekarteringen/'
opgeschoonde_shapefiles_prefix = '../testing/opgeschoonde_shapefiles/'
output_prefix = '../testing/omgezette_vegetatiekarteringen/'

assert Path(originele_karteringen_prefix).exists(), \
    f"De map {originele_karteringen_prefix} bestaat niet. Zet hier de vegetatiekarteringen uit de onedrive in."

assert Path(opgeschoonde_shapefiles_prefix).exists(), \
    f"De map {opgeschoonde_shapefiles_prefix} bestaat niet. Maak deze eerst aan met het handmatig_opschonen_shapefiles.ipynb notebook."

start_time = pd.Timestamp.now()

/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.11.2-CAPI-1.17.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


 ### Omzetclasses instantieren

In [2]:
path_in_wwl = Path("../data/5. Was-wordt-lijst-vegetatietypen-en-habitattypen-09-02-2021.xlsx")
path_out_wwl = Path("../testing/opgeschoonde_waswordt.xlsx")
opschonen_waswordtlijst(path_in_wwl, path_out_wwl)
wwl = WasWordtLijst.from_excel(path_out_wwl)

path_in_dt = Path("../data/definitietabel habitattypen (versie 24 maart 2009)_0.xls")
path_in_mitsjson = Path("../data/mitsjson.json")
path_in_mozaiekjson = Path("../data/mozaiekjson.json")
path_out_dt = Path("../testing/opgeschoonde_definitietabel.xlsx")
opschonen_definitietabel(path_in_dt, path_in_mitsjson, path_in_mozaiekjson, path_out_dt)
dt = DefinitieTabel.from_excel(path_out_dt)

fgr = FGR(Path("../data/bronbestanden/FGR.json"))


/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


Alle SBB codes zijn valide
Alle VvN codes zijn valide
Alle SBB codes zijn valide
Alle VvN codes zijn valide
Alle SBB codes zijn valide
Alle VvN codes zijn valide


 ### Inladen benodigde informatie

 **Overzicht_vegetatiekarteringen.xlsx** is uit de drive en weet van karteringen de provincie, naam, shapefile pad en of het een access of een shapefile kartering is.

 **accesskarteringinfo.xlsx** bevat voor de access karteringen de informatie die nodig is om een access kartering in te laden.

 **shapefilekarteringinfo.xlsx** bevat voor de shapefile karteringen de informatie die nodig is om een shapefile kartering in te laden.


In [3]:
# info over de karteringen
overzicht = pd.read_excel(
    "../data/Overzicht_vegetatiekarteringen.xlsx", 
    sheet_name="Vegetatiekarteringen", 
    engine='openpyxl',
    usecols=[
        "provincie", 
        "naam_kartering", 
        "path_shapes", 
        "landelijke_typologie beschikbaar?", 
        "locatie_landelijke_typologie",
    ],
    dtype="string"
)

# benodigde info voor de access karteringen
access_info = pd.read_excel(
    "../data/accesskarteringinfo.xlsx",
    engine='openpyxl',
    usecols=[
        "naam_kartering",
        "path_csvs",
        "elm_id_name",
    ],
    dtype="string"
)

# benodigde info voor de shapefile karteringen
shape_info = pd.read_excel(
    "../data/shapefilekarteringinfo.xlsx",
    engine='openpyxl',
    usecols=[
        "naam_kartering",
        "ElmID_col",
        "datum_col",
        "opmerking_col",
        "vegtype_col_format",
        "sbb_of_vvn",
        "SBB_col",
        "VvN_col",
        "split_char",
        "perc_col",
    ],
    dtype="string"
)
# Omzetten van pd.NA naar None om te voorkomen dat we overal "if not pd.isna()" moeten doen
# NOTE: dit naar vegkaartering.py?
shape_info = shape_info.applymap(lambda x: None if pd.isna(x) else x)

overzicht = overzicht[overzicht["landelijke_typologie beschikbaar?"] == "Ja"]
# omzetten van ./PROVINCIE/gebied/gebied.shp naar ../testing/opgeschoonde_shapefiles/PROVINCIE/gebied/gebied.shp
overzicht.path_shapes = overzicht.path_shapes.apply(lambda x: opgeschoonde_shapefiles_prefix + x[2:])

access_overzicht = overzicht[overzicht["locatie_landelijke_typologie"] == "access"]
access_overzicht = access_overzicht.join(access_info.set_index("naam_kartering"), on="naam_kartering")
# omzetten van ./PROVINPCIE/gebied/gebied.shp naar ../testing/vegetatiekarteringen/PROVINCIE/gebied/gebied.shp
access_overzicht.path_csvs = access_overzicht.path_csvs.apply(lambda x: originele_karteringen_prefix + x[2:])

shapefile_overzicht = overzicht[overzicht["locatie_landelijke_typologie"] == "in shapes"]
shapefile_overzicht = shapefile_overzicht.join(shape_info.set_index("naam_kartering"), on="naam_kartering")


In [4]:
shapefile_overzicht

,provincie,naam_kartering,path_shapes,landelijke_typologie beschikbaar?,locatie_landelijke_typologie,ElmID_col,datum_col,opmerking_col,vegtype_col_format,sbb_of_vvn,SBB_col,VvN_col,split_char,perc_col
0,Drenthe,Asserbos2015,../testing/opgeschoonde_shapefiles/DR/Asserbos...,Ja,in shapes,OBJECTID,DATE,None,single,VvN,None,CODE_VvN,None,None
1,Drenthe,BG_Drenthe2014,../testing/opgeschoonde_shapefiles/DR/BG_Drent...,Ja,in shapes,None,JAAR,None,single,beide,CODE_SBB,CODE_VvN,None,None
2,Drenthe,BG_Holtingerveld_bossen2016,../testing/opgeschoonde_shapefiles/DR/BG_Holti...,Ja,in shapes,OBJECTID_1,DATUM,OPMERKING,multi,beide,F1_SBB+F2_SBB+F3_SBB+F4_SBB,F1_VvN+F2_VvN+F3_VvN+F4_VvN,+,F1_Perc+F2_Perc+F3_Perc+F4_Perc
3,Drenthe,BG_Hondsrug2019,../testing/opgeschoonde_shapefiles/DR/BG_Honds...,Ja,in shapes,Id,Datum,None,multi,beide,SBBTYPE1+SBBTYPE2,VvN1+VvN2,+,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44,Drenthe extra,NM_Mantingerbos_2015,../testing/opgeschoonde_shapefiles/DR Extra/NM...,Ja,in shapes,None,DATUM,OPM_VELD,multi,beide,F1_SBB1+F2_SBB1+F3_SBB1+F4_SBB1+F5_SBB1,F1_VVN1+F2_VVN1+F3_VVN1+F4_VVN1+F5_VVN1,+,F1_PERC+F2_PERC+F3_PERC+F4_PERC+F5_PERC
48,Friesland,Mandefjild_2015,../testing/opgeschoonde_shapefiles/FR/Bakkevee...,Ja,in shapes,VLAKNR_DEF,None,OPM,multi,beide,1_SBB1+2_SBB1+3_SBB1,1_VVN1+2_VVN1+3_VVN1,+,1_PERC+2_PERC+3_PERC
57,Friesland,KetlikerSkar_2016,../testing/opgeschoonde_shapefiles/FR/Ketliker...,Ja,in shapes,ELMID,Jaar,None,single,VvN,None,VegCode,None,None
62,Friesland,Tsjonger_Middenloop,../testing/opgeschoonde_shapefiles/FR/Tsjonger...,Ja,in shapes,ELMID,DATUM,OPMERKING,multi,SBB,Sbbcatal1+Sbbcatal2,None,+,None


### Karteringen inladen

Hier worden, adhv configuraties uit de spreadsheets, alle karteringen ingeladen. Als er een probleem is met een kartering, wordt deze overgeslagen en wordt een melding gegeven.

Er zijn vier karteringen die op het moment niet ondersteund worden. 
- NM_Leggelderveld2020
  - Vegetatietype codes zijn rVvN
- NM_Mantingerveld2020
  - Vegetatietype codes zijn rVvN
- NM_Norg_heiden2020
  - Vegetatietype codes zijn rVvN
- NM vegetatiekartering RuitenAa2020
  - Er is geen 1 op 1 relatie voor ElmID in de shapefile en in de database

In [5]:
karteringen = {}
mislukte_karteringen = []

for row in access_overzicht.itertuples():
    print(f"------ Inlezen {row.naam_kartering}")
    try:
        karteringen[row.naam_kartering] = Kartering.from_access_db(Path(row.path_shapes), row.elm_id_name, Path(row.path_csvs))
        
    except Exception as e:
        print(e)
        print(f"!!! {row.naam_kartering} wordt overgeslagen !!!")
        mislukte_karteringen.append(row.naam_kartering)
        continue

    print(f"Gelukt!")

for row in shapefile_overzicht.itertuples():
    print(f"------ Inlezen {row.naam_kartering}")
    try:
        karteringen[row.naam_kartering] = Kartering.from_shapefile(
            Path(row.path_shapes),
            vegtype_col_format=row.vegtype_col_format,
            sbb_of_vvn=row.sbb_of_vvn,
            ElmID_col=row.ElmID_col,
            VvN_col=row.VvN_col,
            SBB_col=row.SBB_col, 
            split_char=row.split_char, 
            datum_col=row.datum_col, 
            opmerking_col=row.opmerking_col,
            perc_col=row.perc_col  
        )
    
    except Exception as e:
        print(e)
        print(f"!!! {row.naam_kartering} wordt overgeslagen !!!")
        mislukte_karteringen.append(row.naam_kartering)
        continue

    print(f"Gelukt!")

if len(mislukte_karteringen) > 0:
    print(f"!!! De volgende karteringen zijn niet ingelezen !!!")
    print(mislukte_karteringen)


------ Inlezen GL_Zuidlaardermeer2019
mdbtools v1.0.0
Gelukt!
------ Inlezen NM_Fochteloerveen2013_14
mdbtools v1.0.0
Gelukt!
------ Inlezen NM_Fochteloerveen_2020
mdbtools v1.0.0
Gelukt!
------ Inlezen NM_Klencke2018
mdbtools v1.0.0
Gelukt!
------ Inlezen NM_Leggelderveld2020
mdbtools v1.0.0
Invalid SBB code: r14rg5
!!! NM_Leggelderveld2020 wordt overgeslagen !!!
------ Inlezen NM_Mantingerveld2020
mdbtools v1.0.0
Invalid SBB code: r16rg23
!!! NM_Mantingerveld2020 wordt overgeslagen !!!
------ Inlezen NM_Norg_heiden2020
mdbtools v1.0.0
Invalid SBB code: r50a
!!! NM_Norg_heiden2020 wordt overgeslagen !!!
------ Inlezen NM_NorgerBeekdal2018
mdbtools v1.0.0
Gelukt!
------ Inlezen NM_SBB_Fochteloerveen_randen2016
mdbtools v1.0.0
Gelukt!
------ Inlezen NM_DCR_2013
mdbtools v1.0.0


Gelukt!
------ Inlezen NM_Westerbork2017
mdbtools v1.0.0
Gelukt!
------ Inlezen SBB0756_Drentsche_A2009
mdbtools v1.0.0
Gelukt!
------ Inlezen SBB0757_De_Velden2009
mdbtools v1.0.0
Gelukt!
------ Inlezen SBB0763_Terhorst2009
mdbtools v1.0.0
Gelukt!
------ Inlezen SBB0816_Norg2010
mdbtools v1.0.0
Gelukt!
------ Inlezen SBB0817_Grolloerveen2013
mdbtools v1.0.0
Gelukt!
------ Inlezen SBB0863_Elperstroom_Schoonloo2012
mdbtools v1.0.0
Gelukt!
------ Inlezen SBB0881_SBB_Ruinen_Havelterberg2013
mdbtools v1.0.0
Gelukt!
------ Inlezen SBB0935_DrentsFrieseWold2015
mdbtools v1.0.0
Gelukt!
------ Inlezen SBB0936_DrentseAa2015_16


mdbtools v1.0.0
Gelukt!
------ Inlezen SBB1034_Dwingelderveld2017
mdbtools v1.0.0
Gelukt!
------ Inlezen SBB1039_Drenthe2019
mdbtools v1.0.0
Gelukt!
------ Inlezen Drouwenerzand_2020
mdbtools v1.0.0
Gelukt!
------ Inlezen Fr 0752_Bakkefean_2009


mdbtools v1.0.0
Gelukt!
------ Inlezen Slotplaats_2016
mdbtools v1.0.0
Gelukt!
------ Inlezen Ameland_2010


mdbtools v1.0.0
Gelukt!
------ Inlezen Schiermonnikoog_2015
mdbtools v1.0.0
Gelukt!
------ Inlezen Schiermonnikoog_2017
mdbtools v1.0.0
Gelukt!
------ Inlezen Terschelling_2012


mdbtools v1.0.0
Gelukt!
------ Inlezen Vlieland_2013
mdbtools v1.0.0
Gelukt!
------ Inlezen Lauwersmeer_Fryslan_2015
mdbtools v1.0.0
Gelukt!
------ Inlezen Rottige_Meenthe_Brandemeer_2013
mdbtools v1.0.0
Gelukt!
------ Inlezen Sneekermeergebied_2009
mdbtools v1.0.0
Gelukt!
------ Inlezen Sneekermeergebied_2009_Terkaplester puollen
mdbtools v1.0.0
Gelukt!
------ Inlezen Bakkefean_2022
mdbtools v1.0.0
Gelukt!
------ Inlezen Snitsermar_2021
mdbtools v1.0.0
Gelukt!
------ Inlezen Deelen_2022
mdbtools v1.0.0
Gelukt!
------ Inlezen Tjonger_2021
mdbtools v1.0.0


Gelukt!
------ Inlezen Witte en zwarte brekken_2021
mdbtools v1.0.0


Gelukt!
------ Inlezen NM vegetatiekartering RuitenAa2020
mdbtools v1.0.0
Er is geen 1 op 1 relatie tussen Elmid in de shapefile en ElmID in de Element.csv. Er zitten 3 dubbelingen in de shapefile, bijvoorbeeld Elmid: [8174, 11023, 11051]. Er zitten 3 dubbelingen in Element.csv, bijvoorbeeld ElmID: [8174, 11023, 11051]. 
!!! NM vegetatiekartering RuitenAa2020 wordt overgeslagen !!!
------ Inlezen SBB Duurswold2013
mdbtools v1.0.0
Gelukt!
------ Inlezen SBB Lauwersmeer 2015
mdbtools v1.0.0
Gelukt!
------ Inlezen SBB Westerwolde 2020
mdbtools v1.0.0
Gelukt!
------ Inlezen SBB ZWK 2010 0814_Tussen de Gasten 2010
mdbtools v1.0.0
Gelukt!
------ Inlezen SBB ZWK 2010 0815_Trimunt_2010
mdbtools v1.0.0
Gelukt!
------ Inlezen SGL Hunzedal en Leekstermeer2021 Hunzedal
mdbtools v1.0.0
Gelukt!
------ Inlezen SGL Hunzedal en Leekstermeer2021 Leekstermeer 2021
mdbtools v1.0.0
Gelukt!
------ Inlezen SGL Zuidlaardermeer 2019
mdbtools v1.0.0
Gelukt!
------ Inlezen Asserbos2015
Gelukt!
------ Inlezen BG_

Gelukt!
------ Inlezen NM_Mantingerweiden2010
Gelukt!
------ Inlezen Bosgroep_SNLMonitoring_2014
Gelukt!
------ Inlezen PRV_Witterdiep2021
Gelukt!
------ Inlezen SBB0982_Wijster2017


Gelukt!
------ Inlezen SBB1014_SBB_Kop_van_Drenthe2018
Gelukt!
------ Inlezen SBB1015_ HartvanDrenthe2018
Gelukt!
------ Inlezen SBB1471_DrentsFrieseWold2008
Gelukt!
------ Inlezen NM_Mantingerbos_2015
Gelukt!
------ Inlezen Mandefjild_2015
Gelukt!
------ Inlezen KetlikerSkar_2016
Gelukt!
------ Inlezen Tsjonger_Middenloop
Gelukt!
!!! De volgende karteringen zijn niet ingelezen !!!
['NM_Leggelderveld2020', 'NM_Mantingerveld2020', 'NM_Norg_heiden2020', 'NM vegetatiekartering RuitenAa2020']


 ### Karteringen omzetten

In [6]:
mislukte_karteringen = []

for key, kartering in karteringen.items():
    print(f"------ Waswordtlijst toepassen {key}")
    try:
        kartering.apply_wwl(wwl)    
    except Exception as e:
        print(e)
        print(f"!!! Fout bij {key} !!!")
        mislukte_karteringen.append(key)
        continue
    print(f"Gelukt!")

if len(mislukte_karteringen) > 0:
    print(f"!!! De volgende karteringen hebben een fout bij het toepassen van de was-wordt-lijst !!!")
    print(mislukte_karteringen)


------ Waswordtlijst toepassen GL_Zuidlaardermeer2019
Gelukt!
------ Waswordtlijst toepassen NM_Fochteloerveen2013_14
Gelukt!
------ Waswordtlijst toepassen NM_Fochteloerveen_2020
Gelukt!
------ Waswordtlijst toepassen NM_Klencke2018
Gelukt!
------ Waswordtlijst toepassen NM_NorgerBeekdal2018
Gelukt!
------ Waswordtlijst toepassen NM_SBB_Fochteloerveen_randen2016
Gelukt!
------ Waswordtlijst toepassen NM_DCR_2013
Gelukt!
------ Waswordtlijst toepassen NM_Westerbork2017
Gelukt!
------ Waswordtlijst toepassen SBB0756_Drentsche_A2009
Gelukt!
------ Waswordtlijst toepassen SBB0757_De_Velden2009
Gelukt!
------ Waswordtlijst toepassen SBB0763_Terhorst2009
Gelukt!
------ Waswordtlijst toepassen SBB0816_Norg2010
Gelukt!
------ Waswordtlijst toepassen SBB0817_Grolloerveen2013
Gelukt!
------ Waswordtlijst toepassen SBB0863_Elperstroom_Schoonloo2012
Gelukt!
------ Waswordtlijst toepassen SBB0881_SBB_Ruinen_Havelterberg2013
Gelukt!
------ Waswordtlijst toepassen SBB0935_DrentsFrieseWold2015
Gelukt

/home/jordydelange/git/veg2hab/veg2hab/vegkartering.py:932: UserWarning: Er zijn al VvN aanwezig in de kartering. De was-wordt lijst wordt niet toegepast.
  "Er zijn al VvN aanwezig in de kartering. De was-wordt lijst wordt niet toegepast."


Gelukt!
------ Waswordtlijst toepassen SBB1014_SBB_Kop_van_Drenthe2018
Gelukt!
------ Waswordtlijst toepassen SBB1015_ HartvanDrenthe2018
Gelukt!
------ Waswordtlijst toepassen SBB1471_DrentsFrieseWold2008
Gelukt!
------ Waswordtlijst toepassen NM_Mantingerbos_2015
Gelukt!
------ Waswordtlijst toepassen Mandefjild_2015
Gelukt!
------ Waswordtlijst toepassen KetlikerSkar_2016
Gelukt!
------ Waswordtlijst toepassen Tsjonger_Middenloop
Gelukt!


In [7]:
mislukte_karteringen = []

for key, kartering in karteringen.items():
    print(f"------ Definitietabel toepassen {key}")
    try:  
        kartering.apply_deftabel(dt)        
    except Exception as e:
        print(e)
        print(f"!!! Fout bij {key} !!!")
        mislukte_karteringen.append(key)
        continue
    print(f"Gelukt!")

if len(mislukte_karteringen) > 0:
    print(f"!!! De volgende karteringen hebben een fout bij het toepassen van de definitietabel!!!")
    print(mislukte_karteringen)


------ Definitietabel toepassen GL_Zuidlaardermeer2019
Gelukt!
------ Definitietabel toepassen NM_Fochteloerveen2013_14
Gelukt!
------ Definitietabel toepassen NM_Fochteloerveen_2020
Gelukt!
------ Definitietabel toepassen NM_Klencke2018
Gelukt!
------ Definitietabel toepassen NM_NorgerBeekdal2018
Gelukt!
------ Definitietabel toepassen NM_SBB_Fochteloerveen_randen2016
Gelukt!
------ Definitietabel toepassen NM_DCR_2013
Gelukt!
------ Definitietabel toepassen NM_Westerbork2017
Gelukt!
------ Definitietabel toepassen SBB0756_Drentsche_A2009
Gelukt!
------ Definitietabel toepassen SBB0757_De_Velden2009
Gelukt!
------ Definitietabel toepassen SBB0763_Terhorst2009
Gelukt!
------ Definitietabel toepassen SBB0816_Norg2010
Gelukt!
------ Definitietabel toepassen SBB0817_Grolloerveen2013
Gelukt!
------ Definitietabel toepassen SBB0863_Elperstroom_Schoonloo2012
Gelukt!
------ Definitietabel toepassen SBB0881_SBB_Ruinen_Havelterberg2013
Gelukt!
------ Definitietabel toepassen SBB0935_DrentsFries

In [8]:
mislukte_karteringen = []

for key, kartering in karteringen.items():
    print(f"------ Mitsen/Mozaiekregels checken {key}")
    try:
        kartering.bepaal_habitatkeuzes(fgr)    
    except Exception as e:
        print(e)
        print(f"!!! Fout bij {key} !!!")
        mislukte_karteringen.append(key)
        continue
    print(f"Gelukt!")

if len(mislukte_karteringen) > 0:
    print(f"!!! De volgende karteringen hebben een fout bij het toepassen checken van de criteria!!!")
    print(mislukte_karteringen)


------ Mitsen/Mozaiekregels checken GL_Zuidlaardermeer2019
Iteratie 0: van 2327 naar 39 keuzes nog te bepalen
Iteratie 1: van 39 naar 5 keuzes nog te bepalen
Iteratie 2: van 5 naar 5 keuzes nog te bepalen


/home/jordydelange/git/veg2hab/veg2hab/vegkartering.py:1083: UserWarning: Er zijn nog 5 habitatkeuzes die niet bepaald konden worden.
  f"Er zijn nog {n_keuzes_still_to_determine_post} habitatkeuzes die niet bepaald konden worden."


Gelukt!
------ Mitsen/Mozaiekregels checken NM_Fochteloerveen2013_14
Iteratie 0: van 7940 naar 14 keuzes nog te bepalen
Iteratie 1: van 14 naar 12 keuzes nog te bepalen
Iteratie 2: van 12 naar 12 keuzes nog te bepalen


/home/jordydelange/git/veg2hab/veg2hab/vegkartering.py:1083: UserWarning: Er zijn nog 12 habitatkeuzes die niet bepaald konden worden.
  f"Er zijn nog {n_keuzes_still_to_determine_post} habitatkeuzes die niet bepaald konden worden."


Gelukt!
------ Mitsen/Mozaiekregels checken NM_Fochteloerveen_2020
Iteratie 0: van 3361 naar 5 keuzes nog te bepalen
Iteratie 1: van 5 naar 3 keuzes nog te bepalen
Iteratie 2: van 3 naar 3 keuzes nog te bepalen


...

Gelukt!
------ Mitsen/Mozaiekregels checken NM_Mantingerbos_2015
Iteratie 0: van 149 naar 1 keuzes nog te bepalen
Iteratie 1: van 1 naar 0 keuzes nog te bepalen
Gelukt!
------ Mitsen/Mozaiekregels checken Mandefjild_2015
Iteratie 0: van 313 naar 8 keuzes nog te bepalen
Iteratie 1: van 8 naar 7 keuzes nog te bepalen
Iteratie 2: van 7 naar 7 keuzes nog te bepalen


/home/jordydelange/git/veg2hab/veg2hab/vegkartering.py:1083: UserWarning: Er zijn nog 7 habitatkeuzes die niet bepaald konden worden.
  f"Er zijn nog {n_keuzes_still_to_determine_post} habitatkeuzes die niet bepaald konden worden."


Gelukt!
------ Mitsen/Mozaiekregels checken KetlikerSkar_2016
Iteratie 0: van 1300 naar 0 keuzes nog te bepalen
Gelukt!
------ Mitsen/Mozaiekregels checken Tsjonger_Middenloop
Iteratie 0: van 408 naar 9 keuzes nog te bepalen
Iteratie 1: van 9 naar 7 keuzes nog te bepalen
Iteratie 2: van 7 naar 7 keuzes nog te bepalen


/home/jordydelange/git/veg2hab/veg2hab/vegkartering.py:1083: UserWarning: Er zijn nog 7 habitatkeuzes die niet bepaald konden worden.
  f"Er zijn nog {n_keuzes_still_to_determine_post} habitatkeuzes die niet bepaald konden worden."


Gelukt!


 ### Karteringen wegschrijven

In [9]:
for gebied in (karteringen.keys()):
    print(gebied)
    # Vervangen van de shapefile path naar de output path
    p = Path(overzicht[overzicht.naam_kartering == gebied].path_shapes.str.replace(opgeschoonde_shapefiles_prefix, output_prefix).iloc[0])
    p.parent.mkdir(parents=True, exist_ok=True)
    karteringen[gebied].final_format_to_file(p)


GL_Zuidlaardermeer2019


NM_Fochteloerveen2013_14


NM_Fochteloerveen_2020


NM_Klencke2018


NM_NorgerBeekdal2018


...

SBB1015_ HartvanDrenthe2018
SBB1471_DrentsFrieseWold2008


NM_Mantingerbos_2015


Mandefjild_2015


KetlikerSkar_2016


Tsjonger_Middenloop


In [10]:
end_time = pd.Timestamp.now()
secs = (end_time - start_time).total_seconds()
print(f"Totale tijd: {secs:.2f} seconden.")

# tel aantal vlakken
n_vlakken = 0
for kartering in karteringen.values():
    n_vlakken += kartering.gdf.shape[0]

# tel aantal keuzes
n_keuzes = 0
for kartering in karteringen.values():
    n_keuzes += kartering.gdf.HabitatKeuze.apply(len).sum()

print(f"Er zijn {n_vlakken} vlakken en {n_keuzes} keuzes omgezet.")

vlakken_per_seconde = n_vlakken / secs
keuzes_per_seconde = n_keuzes / secs

print(f"Dat is {vlakken_per_seconde:.2f} vlakken en {keuzes_per_seconde:.2f} keuzes per seconde.")

seconden_per_kartering = secs / len(karteringen)

print(f"Gemiddeld is dat {seconden_per_kartering:.2f} seconden per kartering.")

Totale tijd: 1789.05 seconden.
Er zijn 79337 vlakken en 154282 keuzes omgezet.
Dat is 44.35 vlakken en 86.24 keuzes per seconde.
Gemiddeld is dat 26.31 seconden per kartering.
